In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import email
import re 
import spacy
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rafael.pineda/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rafael.pineda/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Data Loading

In [6]:
df = pd.read_csv("emails.csv", low_memory=False)

In [7]:
# create list of email objects
emails = list(map(email.parser.Parser().parsestr,df['message']))

# extract headings such as subject, from, to etc..
headings  = emails[0].keys()

# Goes through each email and grabs info for each key
# doc['From'] grabs who sent email in all emails
for key in headings:
    df[key] = [doc[key] for doc in emails]

In [19]:
#df.to_csv('emails_parsed.csv')
df = pd.read_csv("emails_parsed.csv", low_memory=False)

In [20]:
df.head()

,Unnamed: 0,file,message,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName
0,0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,<18782981.1075855378110.JavaMail.evans@thyme>,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,NaN,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst
1,1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,<15464986.1075855378456.JavaMail.evans@thyme>,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst
2,2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,<24216240.1075855687451.JavaMail.evans@thyme>,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf
3,3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,<13505866.1075863688222.JavaMail.evans@thyme>,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,NaN,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf
4,4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,<30922949.1075863688243.JavaMail.evans@thyme>,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf


In [21]:
emails = list(map(email.parser.Parser().parsestr,df['message']))

def get_raw_text(emails):
    email_text = []
    for email in emails.walk():
        if email.get_content_type() == 'text/plain':
            email_text.append(email.get_payload())
    return ''.join(email_text)

df['body'] = list(map(get_raw_text, emails))
df.head()
df['user'] = df['file'].map(lambda x: x.split('/')[0])

df['Date'] = pd.to_datetime(df['Date'])
df.head()
df.dtypes


Unnamed: 0                     int64
file                          object
message                       object
Message-ID                    object
Date                          object
From                          object
To                            object
Subject                       object
Mime-Version                 float64
Content-Type                  object
Content-Transfer-Encoding     object
X-From                        object
X-To                          object
X-cc                          object
X-bcc                         object
X-Folder                      object
X-Origin                      object
X-FileName                    object
body                          object
user                          object
dtype: object

In [22]:
print(df['body'].str.contains('- Forwarded by ').sum())

103782


In [24]:
def clean_column(data):
    if data is not None:
        stopwords_list = stopwords.words('english')
        #exclusions = ['RE:', 'Re:', 're:']
        #exclusions = '|'.join(exclusions)
        data =  data.lower()
        data = re.sub('re:', '', data)
        data = re.sub('-', '', data)
        data = re.sub('_', '', data)
        # Remove data between square brackets
        data =re.sub('\[[^]]*\]', '', data)
        # removes punctuation
        data = re.sub(r'[^\w\s]','',data)
        data = re.sub(r'\n',' ',data)
        data = re.sub(r'[0-9]+','',data)
        # strip html 
        p = re.compile(r'<.*?>')
        data = re.sub(r"\'ve", " have ", data)
        data = re.sub(r"can't", "cannot ", data)
        data = re.sub(r"n't", " not ", data)
        data = re.sub(r"I'm", "I am", data)
        data = re.sub(r" m ", " am ", data)
        data = re.sub(r"\'re", " are ", data)
        data = re.sub(r"\'d", " would ", data)
        data = re.sub(r"\'ll", " will ", data)
        data = re.sub('forwarded by phillip k allenhouect on    pm', '',data)
        data = re.sub(r"httpitcappscorpenroncomsrrsauthemaillinkaspidpage", "", data)
        
        data = p.sub('', data)
        if 'forwarded by:' in data:
            data = data.split('subject')[1]
        data = data.strip()
        return data
    return 'No Subject'


df['body_clean'] = df['body'].apply(clean_column)

In [25]:
df['body_clean'].head()

0                                 here is our forecast
1    traveling to have a business meeting takes the...
2                           test successful  way to go
3    randy   can you send me a schedule of the sala...
4                            lets shoot for tuesday at
Name: body_clean, dtype: object

In [30]:
print(" Shape of dataframe: ", df['body_clean'].shape)

 Shape of dataframe:  (517401,)


In [31]:
df.drop_duplicates()
print(" Shape of dataframe after dropping duplicates: ", df.shape)

 Shape of dataframe after dropping duplicates:  (517401, 21)


In [33]:
#Null values

null= df.isnull().sum().sort_values(ascending=False)
total =df.shape[0]
percent_missing= (df.isnull().sum()/total).sort_values(ascending=False)

missing_data= pd.concat([null, percent_missing], axis=1, keys=['Total missing', 'Percent missing'])

missing_data.reset_index(inplace=True)
missing_data= missing_data.rename(columns= { "index": " column name"})
 
print ("Null Values in each column:\n", missing_data)

Null Values in each column:
                   column name  Total missing  Percent missing
0                       X-bcc         517226         0.999662
1                        X-cc         388515         0.750897
2                          To          21847         0.042225
3                     Subject          19187         0.037083
4                        X-To           9153         0.017690
5                  X-FileName           4172         0.008063
6   Content-Transfer-Encoding             29         0.000056
7                    X-Origin             29         0.000056
8                Mime-Version             29         0.000056
9                Content-Type             29         0.000056
10                   X-Folder             29         0.000056
11                     X-From             29         0.000056
12                       user              0         0.000000
13                       body              0         0.000000
14                 Unnamed: 0            

# Vader Sentiment

In [2]:
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()


In [42]:
df['scores'] = df['body_clean'].apply(lambda review: analyser.polarity_scores(review))
df.head()

,Unnamed: 0,file,message,Message-ID,Date,From,To,Subject,Mime-Version,Content-Type,...,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,body,user,body_clean,scores
0,0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 16:39:00-07:00,phillip.allen@enron.com,tim.belden@enron.com,NaN,1.0,text/plain; charset=us-ascii,...,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Here is our forecast\n\n,allen-p,here is our forecast,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 13:51:00-07:00,phillip.allen@enron.com,john.lavorato@enron.com,Re:,1.0,text/plain; charset=us-ascii,...,John J Lavorato <John J Lavorato/ENRON@enronXg...,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,Traveling to have a business meeting takes the...,allen-p,traveling to have a business meeting takes the...,"{'neg': 0.0, 'neu': 0.893, 'pos': 0.107, 'comp..."
2,2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 03:00:00-07:00,phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,1.0,text/plain; charset=us-ascii,...,Leah Van Arsdall,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,test successful. way to go!!!,allen-p,test successful way to go,"{'neg': 0.0, 'neu': 0.513, 'pos': 0.487, 'comp..."
3,3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 06:13:00-07:00,phillip.allen@enron.com,randall.gay@enron.com,NaN,1.0,text/plain; charset=us-ascii,...,Randall L Gay,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,"Randy,\n\n Can you send me a schedule of the s...",allen-p,randy can you send me a schedule of the sala...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 05:07:00-07:00,phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,1.0,text/plain; charset=us-ascii,...,Greg Piper,NaN,NaN,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,Let's shoot for Tuesday at 11:45.,allen-p,lets shoot for tuesday at,"{'neg': 0.375, 'neu': 0.625, 'pos': 0.0, 'comp..."


In [5]:
#df.to_csv('emails_vader.csv')
df = pd.read_csv("emails_vader.csv", low_memory=False)

In [6]:
df.drop(columns=['file','message','Subject','X-To','X-cc','X-bcc','X-FileName','X-Folder','Content-Transfer-Encoding'],inplace=True,axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0.1  517401 non-null  int64  
 1   Unnamed: 0    517401 non-null  int64  
 2   Message-ID    517401 non-null  object 
 3   Date          517401 non-null  object 
 4   From          517401 non-null  object 
 5   To            495554 non-null  object 
 6   Mime-Version  517372 non-null  float64
 7   Content-Type  517372 non-null  object 
 8   X-From        517372 non-null  object 
 9   X-Origin      517372 non-null  object 
 10  body          517401 non-null  object 
 11  user          517401 non-null  object 
 12  body_clean    517146 non-null  object 
 13  scores        517401 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 55.3+ MB


In [7]:
#df.to_csv('emails_vader_small.csv')

In [23]:
df['compound']  = df['scores'].apply(lambda score_dict: eval(score_dict)['compound'])

In [24]:
def Sentimnt(x):
    if x>= 0.05:
        return "Positive"
    elif x<= -0.05:
        return "Negative"
    else:
        return "Neutral"

df['Sentiment'] = df['compound'].apply(Sentimnt)


df.head()

,Unnamed: 0.1,Unnamed: 0,Message-ID,Date,From,To,Mime-Version,Content-Type,X-From,X-Origin,body,user,body_clean,scores,compound,Sentiment
0,0,0,<18782981.1075855378110.JavaMail.evans@thyme>,2001-05-14 16:39:00-07:00,phillip.allen@enron.com,tim.belden@enron.com,1.0,text/plain; charset=us-ascii,Phillip K Allen,Allen-P,Here is our forecast\n\n,allen-p,here is our forecast,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral
1,1,1,<15464986.1075855378456.JavaMail.evans@thyme>,2001-05-04 13:51:00-07:00,phillip.allen@enron.com,john.lavorato@enron.com,1.0,text/plain; charset=us-ascii,Phillip K Allen,Allen-P,Traveling to have a business meeting takes the...,allen-p,traveling to have a business meeting takes the...,"{'neg': 0.0, 'neu': 0.893, 'pos': 0.107, 'comp...",0.9313,Positive
2,2,2,<24216240.1075855687451.JavaMail.evans@thyme>,2000-10-18 03:00:00-07:00,phillip.allen@enron.com,leah.arsdall@enron.com,1.0,text/plain; charset=us-ascii,Phillip K Allen,Allen-P,test successful. way to go!!!,allen-p,test successful way to go,"{'neg': 0.0, 'neu': 0.513, 'pos': 0.487, 'comp...",0.5859,Positive
3,3,3,<13505866.1075863688222.JavaMail.evans@thyme>,2000-10-23 06:13:00-07:00,phillip.allen@enron.com,randall.gay@enron.com,1.0,text/plain; charset=us-ascii,Phillip K Allen,Allen-P,"Randy,\n\n Can you send me a schedule of the s...",allen-p,randy can you send me a schedule of the sala...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,Neutral
4,4,4,<30922949.1075863688243.JavaMail.evans@thyme>,2000-08-31 05:07:00-07:00,phillip.allen@enron.com,greg.piper@enron.com,1.0,text/plain; charset=us-ascii,Phillip K Allen,Allen-P,Let's shoot for Tuesday at 11:45.,allen-p,lets shoot for tuesday at,"{'neg': 0.375, 'neu': 0.625, 'pos': 0.0, 'comp...",-0.3400,Negative


In [26]:
var_funnel = df.groupby('Sentiment').count()['body_clean'].reset_index().sort_values(by='body_clean',ascending=False)
print(var_funnel)

  Sentiment  body_clean
2  Positive      421212
1   Neutral       50157
0  Negative       45777


In [35]:
from plotly import graph_objects as go
fig = go.Figure(go.Funnelarea(
    text =var_funnel.Sentiment,
    values = var_funnel.body_clean,
    title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
    ))
fig.show()

# Creating a DB

In [36]:
import sqlite3

cnx= sqlite3.connect('emails.db')
df.to_sql(name='emails', con=cnx)

517401